## Урок 3. Парсинг HTML

1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта superjob.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе: 
- Наименование вакансии, 
- Предлагаемую зарплату,
- Ссылку на саму вакансию

In [94]:
from bs4 import BeautifulSoup as bs
import requests
import re

main_link = 'https://hh.ru'
add_link = '/search/vacancy?area=1'
key_word = '&text=Python'
class_ = ['vacancy-serp-item', 'bloko-link HH-LinkModifier', 'vacancy-serp-item__compensation']

def get_link(main_link, add_link, key_word):
    link = main_link + add_link + key_word
    return link

def get_html(link):
    if link[0:4] == 'http':
        html = requests.get(link).text
    else:
        with open(link, 'rb') as f:
            html = f.read()
    return html

def parsing_html(link):
    html =  get_html(link)
    parsed_html = bs(html, 'html.parser')
    return parsed_html

def find_class(html, class_):
    found_class = html.findAll(class_=class_)
    return found_class

def find_item(html, class_, href=False):
    found_item = html.find(class_=class_, href=href)
    return found_item

def get_vacancy(html, class_):
    found_class = find_class(html, class_[0])
    class_info = []
    for i in found_class:
        class_dict = {}
        vacancy = find_item(i, class_[1], href=True)
        class_dict['vacancy'] = vacancy.get_text()
        salary = find_item(i, class_[2])
        class_dict['salary'] = ''.join(re.findall('[0-9-]', salary.get_text())) if salary else '-'
        class_dict['link'] = vacancy['href']
        class_info.append(class_dict)
    return class_info


# У hh.ru не работает GET запрос через requests, поэтому заранее записан HTML-файл в браузере.
# link = get_link(main_link, add_link, key_word)
link = 'hh.html'
parsed_html = parsing_html_page(link)
class_info = get_vacancy(parsed_html, class_)
class_info


[{'vacancy': 'Senior python developer в Трипстер (Python, Django, Celery, Redis, Elasticsearch)',
  'salary': '200000',
  'link': 'https://hh.ru/vacancy/33054998?query=Python'},
 {'vacancy': 'Программист Python',
  'salary': '100000-120000',
  'link': 'https://hh.ru/vacancy/32999518?query=Python'},
 {'vacancy': 'Программист Python',
  'salary': '100000',
  'link': 'https://hh.ru/vacancy/33082158?query=Python'},
 {'vacancy': 'Python Developer',
  'salary': '-',
  'link': 'https://hh.ru/vacancy/32648870?query=Python'},
 {'vacancy': 'Backend Developer (Python, Django)',
  'salary': '180000-220000',
  'link': 'https://hh.ru/vacancy/32690277?query=Python'},
 {'vacancy': 'Программист Python',
  'salary': '90000',
  'link': 'https://hh.ru/vacancy/32719006?query=Python'},
 {'vacancy': 'Разработчик Python Backend',
  'salary': '150000',
  'link': 'https://hh.ru/vacancy/33111826?query=Python'},
 {'vacancy': 'Python Developer',
  'salary': '-',
  'link': 'https://hh.ru/vacancy/32524298?query=Pyth

2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)